In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
from dotenv import load_dotenv
from os import getenv
load_dotenv()

In [ ]:
# database connection info
db_con_dict = dict(
    USER=getenv("MYSQL_ROOT_USER"),
    PASSWORD=getenv("MYSQL_ROOT_PASSWORD"),
    HOST=getenv("HOST"),
    PORT=int(getenv("MYSQL_DOCKER_PORT")),
    DATABASE=getenv("MYSQL_DATABASE")    
)